<a href="https://colab.research.google.com/github/OUCTheoryGroup/colab_demo/blob/master/02_Unsupervised_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unsupervised Image Segmentation. *ICASSP* 2018

**图片无监督语义分割**，作者是东京大学的 Asako Kanezaki ，这里采用的是曾伊言修改的代码。

GITHUB地址：https://github.com/Yonv1943/Unsupervised-Segmentation/tree/master

知乎链接：https://zhuanlan.zhihu.com/p/68528056

原作者的算法要运行30秒左右，这里的代码只需要5秒钟就可以取得相同的效果。


In [0]:
# 首先：下载待处理的图像，这里选择的是 tiger.jpg 这张图
! wget https://raw.githubusercontent.com/Yonv1943/Unsupervised-Segmentation/master/image/tiger.jpg

In [0]:
import os
import time
import cv2
import numpy as np
from skimage import segmentation
import torch
import torch.nn as nn
from matplotlib import pyplot as plt

论文的总体框架如下：

![alt text](https://raw.githubusercontent.com/summitgao/ImageGallery/master/20191116182147.jpg)

完整算法如下：

![](https://raw.githubusercontent.com/summitgao/ImageGallery/master/20191116180641.jpg)

其中，$Net()$ 为作者使用的一个全卷积网络，接收输入图像进行特征提取，该网络由三层卷积组成，具体如下：

|  | kernel | dim | stride | padding | activation |
|:--:|:--:|:--:|:--:|:--:|:--:|
|conv2d| 3x3 | 100 | 1 | 1 | RelU, BatchNorm |
|conv2d| 3x3 | 100 | 1 | 1 | RelU, BatchNorm |
|conv2d| 1x1 | 100 | 1 | 1 | BatchNorm |

为了提高效率，曾伊言对网络进行了改进，使用四层卷积，仿照SENet ，使用3x3与1x1交替，膨胀64 与 压缩32。网络的实现代码如下：


In [0]:
class MyNet(nn.Module):
    def __init__(self, inp_dim, mod_dim1, mod_dim2):
        super(MyNet, self).__init__()

        self.seq = nn.Sequential(
            nn.Conv2d(inp_dim, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim2, mod_dim1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(mod_dim1),
            nn.ReLU(inplace=True),

            nn.Conv2d(mod_dim1, mod_dim2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(mod_dim2),
        )

    def forward(self, x):
        return self.seq(x)

## 1. 初始化参数

train_epoch 指定最大迭代 $2^6 = 64$ 个 epoch；inp_dim指输入图像是3通道； mod_dim1 和 mod_dim2 指网络为 64、32通道交替，因为是对原作者代码进行了修改，因此命名前加了 mod

In [0]:
input_image_path = 'tiger.jpg'
train_epoch = 2 ** 6
inp_dim  = 3
mod_dim1 = 64 
mod_dim2 = 32
gpu_id = 0

# if the label number small than it, break loop
min_label_num = 4
# if the label number small than it, start to show result image.
max_label_num = 256  

start_time0 = time.time()

torch.cuda.manual_seed_all(1943)
np.random.seed(1943)
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)  # choose GPU:0
image = cv2.imread(input_image_path)

## 2. 超像素分割

这里使用了Efficient Graph-Based Image Segmentation - Felzenszwalb （MIT）2004 基于图的超像素分割算法 (简称Felz算法)。具体细节不过多介绍。对于超像素分割，有两个算法，一个是 Felz算法，另一个是 SLIC 算法。论文原作者使用的是 SLIC 算法，曾伊言推荐使用 Felz 算法替代 SLIC 算法。具体原因在知乎专栏里说的比较清楚，这里不再介绍。

In [0]:
seg_map = segmentation.felzenszwalb(image, scale=32, sigma=0.5, min_size=64)
plt.imshow(seg_map) 

seg_map = seg_map.flatten()
seg_lab = [np.where(seg_map == u_label)[0]
            for u_label in np.unique(seg_map)]

上面的代码首先进行超像素分割，分割结果保存在 seg_map 里。一共分割得到 616 个区域，各个区域像素的 index 保存在 seg_lab 数组里。

## 3. 算法训练

超像素分割的结果可以看作是**『预分类』**：相似颜色和纹理的像素保存相同的label。比如例子里的 tiger图，超像素分类得到616个区域，分别分配 0 至 615 的标签。

使用上面提到的CNN，对输入图片进行分类，分类的目标是：使输出的分割结果，每一个超像素内分配的标签一致，训练到收敛。

具体来说，把图像输入CNN得到一个图为 output,在 output 里，每个像素被分配一个 label （因为网络最后一层是32个 feature map，用 argmax 取值最大的那个为 label ，因此，label 的范围是 0 到 31）。统计每个超像素里像素的 label，以数量最多的为目标，放到一个 target 的图里，计划 output 和 target 间的交叉熵损失，然后反向传播。

经过多轮训练，CNN会逐步实现具备相同语义信息的小区块合并，得到大区块。（代码设置里，当最终只剩下4个区域时，会停止迭代。）



In [0]:
'''train init'''
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

tensor = image.transpose((2, 0, 1))
tensor = tensor.astype(np.float32) / 255.0
tensor = tensor[np.newaxis, :, :, :]
tensor = torch.from_numpy(tensor).to(device)

model = MyNet(inp_dim, mod_dim1, mod_dim2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-2, momentum=0.9)

image_flatten = image.reshape((-1, 3))
color_avg = np.random.randint(255, size=(max_label_num, 3))
show = image

'''train loop'''
start_time1 = time.time()
model.train()

for batch_idx in range(train_epoch):
    '''forward'''
    optimizer.zero_grad()
    output = model(tensor)[0]
    output = output.permute(1, 2, 0).view(-1, mod_dim2)
    target = torch.argmax(output, 1)
    im_target = target.data.cpu().numpy()

    '''refine'''
    for inds in seg_lab:
        u_labels, hist = np.unique(im_target[inds], return_counts=True)
        im_target[inds] = u_labels[np.argmax(hist)]

    '''backward'''
    target = torch.from_numpy(im_target)
    target = target.to(device)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    '''show image'''
    un_label, lab_inverse = np.unique(im_target, return_inverse=True, )
    if un_label.shape[0] < max_label_num:  # update show
        img_flatten = image_flatten.copy()
        if len(color_avg) != un_label.shape[0]:
            color_avg = [np.mean(img_flatten[im_target == label], axis=0, dtype=np.int) for label in un_label]
        for lab_id, color in enumerate(color_avg):
            img_flatten[lab_inverse == lab_id] = color
        show = img_flatten.reshape(image.shape)

    print('Loss:', batch_idx, loss.item())
    if len(un_label) < min_label_num:
        break

'''save'''
time1 = time.time() - start_time1
print('TimeUsed: %.2f' % time1)
cv2.imwrite("seg_%s_%ds.jpg" % (input_image_path[6:-4], time1), show)
plt.imshow(show)

## 4. 总结

**曾伊言对算法的理解：** 深度学习CNN在整个无监督语义分割任务中，承担的任务是：对经典机器学习无监督语义分割的细粒度预分类结果进行处理。并在迭代中，逐步对小区块进行合并，最后得到符合人类预期的语义分割结果。

但是，这个方法也有明显的**缺点**：那就是鲁棒性不强，算法受参数影响大（包括梯度下降法的参数，与机器学习的预分类算法的参数），并且算法多次随机重启的结果会有不同。